# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [2]:
# Libraries
import pandas as pd
import numpy as np
import os

In [3]:
os.listdir()

['.ipynb_checkpoints', 'Admission_Predict.csv', 'data', 'main.ipynb']

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [4]:
# your code here
ap =pd.read_csv("Admission_Predict.csv")


Remove trailing spaces at the end of the column names if there are any.


In [5]:
# your code here

ap.columns = ap.columns.str.strip() 
print(ap.columns)

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')


Let's evaluate the dataset by looking at the `head` function.

In [6]:
# your code here
ap.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [7]:
# your code here
print(ap.isnull())
ap.isnull().sum()

     Serial No.  GRE Score  TOEFL Score  University Rating    SOP    LOR  \
0         False      False        False              False  False  False   
1         False      False        False              False  False  False   
2         False      False        False              False  False  False   
3         False      False        False              False  False  False   
4         False      False        False              False  False  False   
..          ...        ...          ...                ...    ...    ...   
380       False      False        False              False  False  False   
381       False      False        False              False  False  False   
382       False      False        False              False  False  False   
383       False      False        False              False  False  False   
384       False      False        False              False  False  False   

      CGPA  Research  Chance of Admit  
0    False     False            False  
1    Fa

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [8]:
# your code here
ap.set_index(["Serial No."])

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...
381,324,110,3,3.5,3.5,9.04,1,0.82
382,325,107,3,3.0,3.5,9.11,1,0.84
383,330,116,4,5.0,4.5,9.45,1,0.91


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [9]:
# your code here
ap["GRE Score"].nunique()
ap["CGPA"].nunique()

ap_str= ap.astype(str)
ap_str["GRE+CGPA"] = ap_str["GRE Score"] + ap_str["CGPA"]

print(ap_str["GRE+CGPA"].nunique())
ap_str.shape

#Its a match!
ap_str

385


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,GRE+CGPA
0,1,337,118,4,4.5,4.5,9.65,1,0.92,3379.65
1,2,316,104,3,3.0,3.5,8.0,1,0.72,3168.0
2,3,322,110,3,3.5,2.5,8.67,1,0.8,3228.67
3,4,314,103,2,2.0,3.0,8.21,0,0.65,3148.21
4,5,330,115,5,4.5,3.0,9.34,1,0.9,3309.34
...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,3249.04
381,382,325,107,3,3.0,3.5,9.11,1,0.84,3259.11
382,383,330,116,4,5.0,4.5,9.45,1,0.91,3309.45
383,384,312,103,3,3.5,4.0,8.78,0,0.67,3128.78


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [10]:
# your code here
ap.set_index(["GRE Score", "CGPA"],drop=True).reset_index(inplace=True)
ap.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [11]:
# your code here 
db1 =ap[(ap["CGPA"]>9) & (ap["Research"]==1)]
db1
#101 rows

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
4,5,330,115,5,4.5,3.0,9.34,1,0.90
10,11,328,112,4,4.0,4.5,9.10,1,0.78
19,20,328,116,5,5.0,5.0,9.50,1,0.94
20,21,334,119,5,5.0,4.5,9.70,1,0.95
...,...,...,...,...,...,...,...,...,...
379,380,329,111,4,4.5,4.0,9.23,1,0.89
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [12]:
# your code here

ap[(ap["CGPA"]> 9) & (ap["SOP"]<3.5)]["Chance of Admit"].mean()

0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [13]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    
    # your code here
    mean = np.mean(col)
    std = np.std(col)
    standardized = (col - mean) / std
    return standardized
standardize(ap["CGPA"])

0      1.750174
1     -0.992501
2      0.121191
3     -0.643433
4      1.234884
         ...   
380    0.736216
381    0.852571
382    1.417729
383    0.304036
384    1.766796
Name: CGPA, Length: 385, dtype: float64

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [14]:
# your code here
ap["CGPA_std"]= standardize(ap["CGPA"])
ap["GRE_std"]= standardize(ap["GRE Score"])
ap["LOR_std"]= standardize(ap["LOR"])

admissions = ap


We will generate the decision choice at random using the code below. Please run the cell.

In [15]:
# Libraries
from random import choices

In [24]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])
admissions["decision_choice"]=decision_choice

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [25]:
# your code here

admissions["deciding_column"]= admissions.lookup(admissions.index, admissions['decision_choice'])
admissions

C:\Users\biggu\AppData\Local\Temp\ipykernel_10584\18427660.py:3: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  admissions["deciding_column"]= admissions.lookup(admissions.index, admissions['decision_choice'])


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision_choice
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.193197,LOR_std
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,-0.992501,CGPA_std
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,-1.039517,LOR_std
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.236698,GRE_std
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,-0.481338,LOR_std
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.736216,0.629546,0.076840,0.076840,LOR_std
381,382,325,107,3,3.0,3.5,9.11,1,0.84,0.852571,0.716170,0.076840,0.852571,CGPA_std
382,383,330,116,4,5.0,4.5,9.45,1,0.91,1.417729,1.149292,1.193197,1.149292,GRE_std
383,384,312,103,3,3.5,4.0,8.78,0,0.67,0.304036,-0.409947,0.635019,0.304036,CGPA_std


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [27]:
# your code here
admissions["decision"] = [1 if x > 0.8 else 0 for x in admissions["deciding_column"]]

admissions

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision_choice,decision
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.193197,LOR_std,1
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,-0.992501,CGPA_std,0
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,-1.039517,LOR_std,0
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.236698,GRE_std,0
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,-0.481338,LOR_std,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.736216,0.629546,0.076840,0.076840,LOR_std,0
381,382,325,107,3,3.0,3.5,9.11,1,0.84,0.852571,0.716170,0.076840,0.852571,CGPA_std,1
382,383,330,116,4,5.0,4.5,9.45,1,0.91,1.417729,1.149292,1.193197,1.149292,GRE_std,1
383,384,312,103,3,3.5,4.0,8.78,0,0.67,0.304036,-0.409947,0.635019,0.304036,CGPA_std,0


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [29]:
# your code here
count = (admissions["decision"] == 1).sum()
count


87

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [38]:
# your code here
import re

a_columns = admissions.columns
a_columns = [re.sub(r'[^a-zA-Z]+', '', column) for column in a_columns]
a_columns = [column.replace(" ", "_") for column in a_columns]
a_columns = [column.lower() for column in a_columns]
admissions.columns = a_columns
admissions

,serialno,grescore,toeflscore,universityrating,sop,lor,cgpa,research,chanceofadmit,cgpastd,grestd,lorstd,decidingcolumn,decisionchoice,decision
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1.750174,1.755663,1.193197,1.193197,LOR_std,1
1,2,316,104,3,3.0,3.5,8.00,1,0.72,-0.992501,-0.063450,0.076840,-0.992501,CGPA_std,0
2,3,322,110,3,3.5,2.5,8.67,1,0.80,0.121191,0.456297,-1.039517,-1.039517,LOR_std,0
3,4,314,103,2,2.0,3.0,8.21,0,0.65,-0.643433,-0.236698,-0.481338,-0.236698,GRE_std,0
4,5,330,115,5,4.5,3.0,9.34,1,0.90,1.234884,1.149292,-0.481338,-0.481338,LOR_std,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82,0.736216,0.629546,0.076840,0.076840,LOR_std,0
381,382,325,107,3,3.0,3.5,9.11,1,0.84,0.852571,0.716170,0.076840,0.852571,CGPA_std,1
382,383,330,116,4,5.0,4.5,9.45,1,0.91,1.417729,1.149292,1.193197,1.149292,GRE_std,1
383,384,312,103,3,3.5,4.0,8.78,0,0.67,0.304036,-0.409947,0.635019,0.304036,CGPA_std,0


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [42]:
# your code here
admissions["grescore"] = admissions["grescore"].apply(lambda x: x + 10 if admissions[admissions["universityrating"] >= 4]["universityrating"].any() else x)

labels=["low", "medium", "high", "excellent"]
adjusted_gre = pd.cut(admissions["grescore"], bins=4, labels=labels)
adjusted_gre

0      excellent
1           high
2           high
3         medium
4      excellent
         ...    
380         high
381         high
382    excellent
383       medium
384    excellent
Name: grescore, Length: 385, dtype: category
Categories (4, object): ['low' < 'medium' < 'high' < 'excellent']